In [1]:
import pandas as pd
predicted_country_df = pd.read_csv("predicted_temp_country.csv")
predicted_country_df

,Unnamed: 0,Country,Year,avg_temp_F
0,0,Afghanistan,2013,59.620016
1,1,Afghanistan,2014,59.669303
2,2,Afghanistan,2015,59.718589
3,3,Afghanistan,2016,59.767875
4,4,Afghanistan,2017,59.817161
...,...,...,...,...
19267,19267,Zimbabwe,2096,73.153647
19268,19268,Zimbabwe,2097,73.175849
19269,19269,Zimbabwe,2098,73.198050
19270,19270,Zimbabwe,2099,73.220251


In [3]:
previous_data = pd.read_csv("country_temp_data_ML.csv")
previous_data = previous_data[["Country","Year","avg_temp_F"]]


,Country,Year,AverageTemperature,AverageTemperatureUncertainty,Month,Day,avg_temp_F,latitude,longitude,year change,running_total_change
0,Afghanistan,1900,13.749333,0.748833,6.5,1.0,56.74880,33.939110,67.709953,0.00000,0.00000
1,Afghanistan,1901,13.894000,0.699333,6.5,1.0,57.00920,33.939110,67.709953,0.26040,0.26040
2,Afghanistan,1902,14.505833,0.726083,6.5,1.0,58.11050,33.939110,67.709953,1.10130,1.36170
3,Afghanistan,1903,12.986417,0.732333,6.5,1.0,55.37555,33.939110,67.709953,-2.73495,-1.37325
4,Afghanistan,1904,13.805750,0.713667,6.5,1.0,56.85035,33.939110,67.709953,1.47480,0.10155
...,...,...,...,...,...,...,...,...,...,...,...
24742,Zimbabwe,2008,21.545583,0.433000,6.5,1.0,70.78205,-19.015438,29.154857,-0.10560,0.48555
24743,Zimbabwe,2009,21.377250,0.436417,6.5,1.0,70.47905,-19.015438,29.154857,-0.30300,0.18255
24744,Zimbabwe,2010,21.986250,0.409667,6.5,1.0,71.57525,-19.015438,29.154857,1.09620,1.27875
24745,Zimbabwe,2011,21.602417,0.393417,6.5,1.0,70.88435,-19.015438,29.154857,-0.69090,0.58785


In [2]:
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

In [ ]:
beg_temp = 0
predicted_country_df["year change"] = 0
predicted_country_df["running_total_change"] = 0

for index, row in predicted_country_df.iterrows():
    if row["Year"] == 2013:
        predicted_country_df.loc[index, 'year change'] = 0
        predicted_country_df.loc[index, 'running_total_change'] = 0
        beg_temp = row["avg_temp_F"]
        running_total = 0
    else:
        change = row["avg_temp_F"] - beg_temp
        running_total = running_total + change
        predicted_country_df.loc[index, 'year change'] = change
        predicted_country_df.loc[index, 'running_total_change'] = running_total
        beg_temp = row["avg_temp_F"]
        
country_df_merged_year.head(20)